In [1]:
# Installing necesarry packages

! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece] scikit-learn

     |████████████████████████████████| 290 kB 4.1 MB/s 
     |████████████████████████████████| 3.1 MB 42.4 MB/s 
     |████████████████████████████████| 90 kB 8.6 MB/s 
     |████████████████████████████████| 1.2 MB 40.6 MB/s 
     |████████████████████████████████| 243 kB 38.1 MB/s 
     |████████████████████████████████| 1.1 MB 38.9 MB/s 
     |████████████████████████████████| 59 kB 7.3 MB/s 
     |████████████████████████████████| 132 kB 53.0 MB/s 
     |████████████████████████████████| 596 kB 38.3 MB/s 
     |████████████████████████████████| 895 kB 40.9 MB/s 
     |████████████████████████████████| 3.3 MB 35.8 MB/s 
     |████████████████████████████████| 160 kB 40.8 MB/s 
     |████████████████████████████████| 271 kB 44.9 MB/s 
     |████████████████████████████████| 192 kB 45.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import os

import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer, MarianMTModel, MarianTokenizer

from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split
import datasets

import random
import numpy as np
import pandas as pd
from IPython.display import display, HTML


os.environ["WANDB_DISABLED"]="true"
print(transformers.__version__)

4.12.5


In [3]:
# All necessary variables
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "zu"
batch_size = 8
epoch = 5
learning_rate=1e-5
weight_decay = 0.001
model_checkpoint = "Helsinki-NLP/opus-mt-en-ROMANCE" # Pretrained Model path
model_name = model_checkpoint.split("/")[-1]

In [ ]:
# Uploading data
base_url = '/content/drive/MyDrive/NLP_Lab/'
raw_datasets = load_dataset('csv', data_files=[base_url + 'en-zu.training.csv'])
eval_dataset = load_dataset('csv', data_files=[base_url + 'en-zu.eval.csv'])
# Preparing data
eval_dataset = eval_dataset.remove_columns('zu.1')
eval_dataset = eval_dataset.rename_column('Source','source')
# Uploading Metrics
metric = load_metric("sacrebleu")


Using custom data configuration default-1a633873e3bb682d


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-1a633873e3bb682d/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-fb68402338783d4b


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-fb68402338783d4b/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Let's visualize a sentence
raw_datasets["train"][0]

{'en': "His solo albums in the 60s were some of the most brilliant and breathtaking beautiful sounds I've ever heard.",
 'source': 'News Crawl 2019',
 'zu': 'Ama-albhamu akhe e-solo eminyaka yawo-60 angamanye anezingoma ezimnandi kakhulu kwengake ngazizwa.'}

In [ ]:
# Let's visualize a sentence
eval_dataset["train"][0]

{'en': 'Peter Van Sant: And it means what?',
 'source': 'News Crawl 2019',
 'zu': 'Peter Van Sant: Bese kusho ukuthini?'}

In [ ]:

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(raw_datasets["train"])

,en,zu,source
0,"""Collaboration with industry partners and customers is needed to construct a new framework to pre-vet legitimate, low-risk travellers.",Ukubonisana nomana bemboni namakhasimende kudingekile ukwakha uhlaka olusha kubahambi abanobucayi obusezingeni eliphansi.,News Crawl 2019
1,He was a talented basketball player as a junior and was picked up by Collingwood with pick No. 32 at the 2015 draft.,Wayengumdlali webhola lezandla onethalente esemncane wabe esethathwe yi-Collingwood ekhethwe ngoNo-32 kuhlaka luka-2015.,News Crawl 2019
2,"""On 7 February 2017 Colin died in a collision with a train, in circumstances we believe were avoidable.","“Ngomhlaka 7 kuNhlolanja washona u-Colin ekushayisaneni nesitimela, esimeni esikholelwa ngukuthi sasingagwemeka.",News Crawl 2019
3,"She had been working seven months on contract at the Perambra Taluk hospital, in the countryside outside Kozhikode (once Calicut).","Ubelokhu esebenza izinyanga eziyisikhombisa kuyinkontileka esibhedlela iPerambra Taluk, emaphandleni angaphandle kweKozhikode (eyake yaba iCalicut).",News Crawl 2019
4,Those wavering Tory MPs who had hoped to postpone their personal moment of truth to give him time to agree a Brexit deal no longer have that luxury.,"Uphelile umbuso walabo ababengabaza amalungu ephalamende ka-Tory, ababengafuni ukukhuluma iqiniso ukumnika isikhathi ukuba avume isivumelwano se-Brexit.",News Crawl 2019


Downloading:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/801k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/794k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

In [ ]:

def preprocess_function(examples):
    '''
    Pre- and post-processing of tokens
    '''
    inputs = [prefix + ex for ex in examples[source_lang]]
    targets = [ex for ex in examples[target_lang]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

def postprocess_text(preds, labels):
    '''
    post processing the outputs for BLEU
    '''
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
    
def compute_metrics(eval_preds):
    '''
    computes BLEU
    '''
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
'''
This will select sentencpiece
'''    

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint) 

In [ ]:
# Let's visualize some token embeddings
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[368, 129, 902, 29306, 17, 15, 4, 1826, 17, 79, 150, 8, 4, 329, 15126, 11, 26085, 2487, 7980, 34, 22, 2008, 828, 1535, 3, 0], [852, 151, 39, 6780, 18, 164, 25, 667, 9, 2179, 4, 6997, 2, 25, 221, 30, 14252, 97, 474, 14680, 9, 238, 1478, 4536, 4200, 38, 2080, 4289, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[4610, 276, 727, 250, 25794, 28575, 2808, 55, 225, 276, 3241, 902, 55, 225, 1832, 7254, 682, 7, 9280, 276, 8127, 4586, 1835, 1197, 13, 1024, 34582, 843, 55, 8955, 4512, 422, 4607, 4461, 8717, 13368, 55, 12410, 4160, 1377, 7455, 8974, 3, 0], [312, 531, 31981, 27, 10783, 173, 1476, 2102, 10683, 4798, 6037, 1018, 15347, 6691, 26336, 295, 1902, 422, 15347, 1287, 4607, 3028, 533, 21036, 422, 2, 295, 1476, 2102, 295, 10783, 7714, 9067, 1018, 15347, 6691, 55, 1476, 425, 1377, 533, 810, 4170, 9280, 55, 8955, 97, 

In [ ]:
#Preparing training and validation data
tokenized_train_dataset = raw_datasets.map(preprocess_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(preprocess_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
# Getting pretrained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/286M [00:00<?, ?B/s]

In [ ]:
'''
This is tuned to en-xh-sentencepiece model. I think comparing on 5 epochs is fine
'''
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=weight_decay,
    save_total_limit=3,
    num_train_epochs=epoch,
    predict_with_generate=True    
)

Using the `WAND_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # initializing data collator

In [ ]:
# Training args
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_train_dataset["train"],
    eval_dataset=tokenized_eval_dataset["train"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: en, zu, source.
***** Running training *****
  Num examples = 4739
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2965


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,5.294200,4.343840,3.236200,57.158300
2,4.480900,4.027222,3.696100,68.378800
3,4.233100,3.865894,3.906700,71.968900
4,4.083700,3.784572,4.012600,72.094200
5,4.001300,3.759354,4.118000,71.762500


Saving model checkpoint to opus-mt-en-sw-finetuned-en-to-zu/checkpoint-500
Configuration saved in opus-mt-en-sw-finetuned-en-to-zu/checkpoint-500/config.json
Model weights saved in opus-mt-en-sw-finetuned-en-to-zu/checkpoint-500/pytorch_model.bin
tokenizer config file saved in opus-mt-en-sw-finetuned-en-to-zu/checkpoint-500/tokenizer_config.json
Special tokens file saved in opus-mt-en-sw-finetuned-en-to-zu/checkpoint-500/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: en, zu, source.
***** Running Evaluation *****
  Num examples = 998
  Batch size = 8
Saving model checkpoint to opus-mt-en-sw-finetuned-en-to-zu/checkpoint-1000
Configuration saved in opus-mt-en-sw-finetuned-en-to-zu/checkpoint-1000/config.json
Model weights saved in opus-mt-en-sw-finetuned-en-to-zu/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in opus-mt-en-sw-finetuned-en-to-zu/checkpoint-1000/token

TrainOutput(global_step=2965, training_loss=4.345925527640177, metrics={'train_runtime': 3350.6701, 'train_samples_per_second': 7.072, 'train_steps_per_second': 0.885, 'total_flos': 1158809513951232.0, 'train_loss': 4.345925527640177, 'epoch': 5.0})

In [ ]:
# Testing our model against google translation
src_text = ['My name is Muhammad and I live in Africa']
model_name = 'opus-mt-en-xh-finetuned-en-to-zu/checkpoint-500'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]


In [ ]:
print("Google translation: \n Igama lami nginguMuhammad futhi ngihlala e-Afrika")